#**Введение**
###**О проекте:**
Работу выполнили студенты 2 курса ИТИС Свидиров Кирилл, Набиев Азат группы 11-902 и 11-906.

###**Задачи проекта:**
На основе данных оценок отелей:
  - 1. Посмотреть на даты первых и последних обновлений.
  - 2. Какие штаты занимают нижние и верхние позиции по среднему рейтингу отзывов об отелях?
  - 3. Сгруппировать по главным категориям и вывести это в график(
    Даст возможность увидеть, на что у нас имеются отзывы. При желании просматривать количество отзывов на конкретную отрасль.
  ).
  - 4. Сгруппировать по городам.
  - 5. Группировка по провинциям, штатам.
  - 6. Найти взаимосвязь между населением штата и количеством отзывов об отелях.
  - 7. Найти взаимосвязь между туристическим бюджетом штата и количеством отзывов об отелях.
  - 8. Узнать по широте и долготе, где находится больше всего отелей(обратить внимание на близость\отдаленность к экватору, морю и курортным зонам).

###**Цель:**

Помочь предпринимателям найти подходящую местность для открытия своего отеля.

###**Источник данных:**
Ссылка : https://www.kaggle.com/datafiniti/hotel-reviews?select=Datafiniti_Hotel_Reviews.csv


#Install extra libs

Для визуализации воспользуемся библиотекой plotly - на первый взгляд она кажется довольна мощной, и сфера её применения не ограничивается python


In [1]:
!pip install chart-studio

     |████████████████████████████████| 71kB 5.0MB/s 


In [2]:
!pip install plotly

#Imports


In [3]:
#-----<XxXxX>-----#

import pandas as pd
import numpy as np
import scipy as sc
import matplotlib.pyplot as plt
from google.colab import drive
%matplotlib inline
from datetime import datetime

#-----<Plotly>-----#
import chart_studio.plotly as py

py.sign_in('OneWayDream', 'R956rptWwcwqhvLztngF')

from plotly.graph_objs import *

import plotly
import plotly.graph_objs as go
import plotly.express as px
from plotly.subplots import make_subplots

#-----<Models>-----#

from sklearn.linear_model import LinearRegression

#-----<XxXxX>-----#

#Получение данных


In [4]:
drive.mount('/content/drive')

Mounted at /content/drive


In [5]:
dataset=pd.read_csv('/content/drive/MyDrive/Reviews_Analysis/Datafiniti_Hotel_Reviews.csv')

#Описание данных


## **Рассмотрим полученные данные:**

In [6]:
dataset.head()

,id,dateAdded,dateUpdated,address,categories,primaryCategories,city,country,keys,latitude,longitude,name,postalCode,province,reviews.date,reviews.dateSeen,reviews.rating,reviews.sourceURLs,reviews.text,reviews.title,reviews.userCity,reviews.userProvince,reviews.username,sourceURLs,websites
0,AVwc252WIN2L1WUfpqLP,2016-10-30T21:42:42Z,2018-09-10T21:06:27Z,5921 Valencia Cir,"Hotels,Hotels and motels,Hotel and motel reser...",Accommodation & Food Services,Rancho Santa Fe,US,us/ca/ranchosantafe/5921valenciacir/359754519,32.990959,-117.186136,Rancho Valencia Resort Spa,92067,CA,2013-11-14T00:00:00Z,"2016-08-03T00:00:00Z,2016-07-26T00:00:00Z,2016...",5.0,https://www.hotels.com/hotel/125419/reviews%20/,Our experience at Rancho Valencia was absolute...,Best romantic vacation ever!!!!,NaN,NaN,Paula,http://www.hotels.com/ho125419/%25252525253Flo...,http://www.ranchovalencia.com
1,AVwc252WIN2L1WUfpqLP,2016-10-30T21:42:42Z,2018-09-10T21:06:27Z,5921 Valencia Cir,"Hotels,Hotels and motels,Hotel and motel reser...",Accommodation & Food Services,Rancho Santa Fe,US,us/ca/ranchosantafe/5921valenciacir/359754519,32.990959,-117.186136,Rancho Valencia Resort Spa,92067,CA,2014-07-06T00:00:00Z,"2016-08-02T00:00:00Z,2016-08-26T00:00:00Z,2016...",5.0,https://www.hotels.com/hotel/125419/reviews%20/,Amazing place. Everyone was extremely warm and...,Sweet sweet serenity,NaN,NaN,D,http://www.hotels.com/ho125419/%25252525253Flo...,http://www.ranchovalencia.com
2,AVwc252WIN2L1WUfpqLP,2016-10-30T21:42:42Z,2018-09-10T21:06:27Z,5921 Valencia Cir,"Hotels,Hotels and motels,Hotel and motel reser...",Accommodation & Food Services,Rancho Santa Fe,US,us/ca/ranchosantafe/5921valenciacir/359754519,32.990959,-117.186136,Rancho Valencia Resort Spa,92067,CA,2015-01-02T00:00:00Z,"2016-11-15T00:00:00Z,2016-08-23T00:00:00Z,2016...",5.0,https://www.hotels.com/hotel/125419/reviews%20/,We booked a 3 night stay at Rancho Valencia to...,Amazing Property and Experience,NaN,NaN,Ron,http://www.hotels.com/ho125419/%25252525253Flo...,http://www.ranchovalencia.com
3,AVwdOclqIN2L1WUfti38,2015-11-28T19:19:35Z,2018-09-10T21:06:16Z,7520 Teague Rd,"Hotels,Hotels and motels,Travel agencies and b...",Accommodation & Food Services,Hanover,US,us/md/hanover/7520teaguerd/-2043779672,39.155929,-76.716341,Aloft Arundel Mills,21076,MD,2016-05-15T00:00:00Z,"2016-05-21T00:00:00Z,2016-07-31T00:00:00Z",2.0,https://www.tripadvisor.com/Hotel_Review-g4118...,Currently in bed writing this for the past hr ...,"Never again...beware, if you want sleep.",Richmond,VA,jaeem2016,http://www.yellowbook.com/profile/aloft-arunde...,http://www.starwoodhotels.com/alofthotels/prop...
4,AVwdOclqIN2L1WUfti38,2015-11-28T19:19:35Z,2018-09-10T21:06:16Z,7520 Teague Rd,"Hotels,Hotels and motels,Travel agencies and b...",Accommodation & Food Services,Hanover,US,us/md/hanover/7520teaguerd/-2043779672,39.155929,-76.716341,Aloft Arundel Mills,21076,MD,2016-07-09T00:00:00Z,2016-07-31T00:00:00Z,5.0,https://www.tripadvisor.com/Hotel_Review-g4118...,I live in Md and the Aloft is my Home away fro...,ALWAYS GREAT STAY...,Laurel,MD,MamaNiaOne,http://www.yellowbook.com/profile/aloft-arunde...,http://www.starwoodhotels.com/alofthotels/prop...


In [7]:
dataset.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10000 entries, 0 to 9999
Data columns (total 25 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   id                    10000 non-null  object 
 1   dateAdded             10000 non-null  object 
 2   dateUpdated           10000 non-null  object 
 3   address               10000 non-null  object 
 4   categories            10000 non-null  object 
 5   primaryCategories     10000 non-null  object 
 6   city                  10000 non-null  object 
 7   country               10000 non-null  object 
 8   keys                  10000 non-null  object 
 9   latitude              10000 non-null  float64
 10  longitude             10000 non-null  float64
 11  name                  10000 non-null  object 
 12  postalCode            10000 non-null  object 
 13  province              10000 non-null  object 
 14  reviews.date          10000 non-null  object 
 15  reviews.dateSeen    

## **Описание полей:**

**id** *(Text)* - идентификатор. Предстоит выяснить, что он идентифицирует( На 10000 записей около 1800 уникальных идентификаторов, так что явно не отзыв

**dateAdded** *(Date)* - время добавления компании в датасет

*dateUpdated** *(Date)* - время последнего обновления информации об компании или наблюдения за ней

**address** *(Text)* - физический почтовый адрес компании

**сategories** *(Text)* - список ключевых слов-категорий для данной компании

**primaryCategories** *(Text)* - список стандартизированных категорий, к которым принадлежит данная компания

```
Возможные значения:

- Accommodation & Food Services
- Administrative Support Waste Management & Remediation
- Agriculture
- Arts Entertainment & Recreation
- Construction
- Educational Services
- Finance & Insurance
- Healthcare & Social Assistance
- Information
- Management of Companies & Enterprises
- Manufacturing
- Mining Quarrying & Oil & Gas Extraction
- Other Services
- Professional Scientific & Technical Services
- Public Administration
- Real Estate Rental & Leasing
- Retail
- Transportation & Warehousing
- Utilities
```
Источник: https://developer.datafiniti.co/docs/possible-values-for-business-fields

**city** *(Text)* - город, в котором расположена компания

**country** *(Text)* - двухбуквенный код страны местонахождения компании

**keys** *(Keyword - Text)* - фактический идентификатор. Значение, генеруруемое из нескольких полей записи, позволяющее избежать дублирования

Подробнее: https://developer.datafiniti.co/docs/how-business-records-are-merged

**latitude** *(Double)* - координаты компании по широте

**longitude** *(Double)* - координаты компании по долготе

**name** *(Text)* - название компании

**postalCode** *(Integer)* - почтовый индекс компании

**province** *(Text)* - провинция или штат нахождения компании

**reviews.date** *(Date)* - дата создания отзыва

**reviews.dateSeen** *(Date)* - даты просмотров этого отзыва

**reviews.rating** *(Integer)* - оценка компании из отзыва

**reviews.sourceURLs** *(Text)* - список сайтов, на которых данный отзыв использовался / упоминался

**reviews.text** *(Text)* - Текст-контент отзыва

**reviews.title** *(Text)* - Заголовок отзыва

**reviews.userCity** *(Text)* - Город пользователя, который оставил отзыв

**reviews.userProvince** *(Text)* - Штат или провинция пользователя

**reviews.username** *(Text)* - Имя пользователя

**sourceURLs** *(Text)* - Список использованных источников для получения этой записи

**websites** *(Text)* - Список веб-сайтов данной компании



Сайт, предоставляющий информацию о значение полей:

https://developer.datafiniti.co/docs/business-data-schema


#Предобработка данных

##Обработка пустых строк

In [8]:
print(dataset.isnull().sum())

id                         0
dateAdded                  0
dateUpdated                0
address                    0
categories                 0
primaryCategories          0
city                       0
country                    0
keys                       0
latitude                   0
longitude                  0
name                       0
postalCode                 0
province                   0
reviews.date               0
reviews.dateSeen           0
reviews.rating             0
reviews.sourceURLs         0
reviews.text               1
reviews.title              1
reviews.userCity        5836
reviews.userProvince    7295
reviews.username           0
sourceURLs                 0
websites                   0
dtype: int64


Поля, содержащие информацию о местоположении пользователя (reviews.userCity и reviews.userProvince) содержат много пропусков. Но так как эти данные могут понадобиться при анализе отзывов, создадим после 2 датасета: со всеми записями и без разреженных полей и с частью записей, но с данными полями.

##Проверка уникальных значений

In [9]:
for col in dataset.columns:
    print(col, len(dataset[col].unique()))

id 1853
dateAdded 1788
dateUpdated 1632
address 1846
categories 1148
primaryCategories 6
city 1021
country 1
keys 1853
latitude 1849
longitude 1850
name 1670
postalCode 1474
province 49
reviews.date 3188
reviews.dateSeen 2053
reviews.rating 30
reviews.sourceURLs 6386
reviews.text 9982
reviews.title 8623
reviews.userCity 2052
reviews.userProvince 178
reviews.username 6942
sourceURLs 1853
websites 1746


Поля country и primaryCategories принимают мало значений (1и 6 соответственно). Рассмотрим эти значения.


In [10]:
print('country', dataset['country'].unique())
print('primaryCategories', dataset['primaryCategories'].unique())

country ['US']
primaryCategories ['Accommodation & Food Services'
 'Healthcare & Social Assistance,Accommodation & Food Services'
 'Accommodation & Food Services,Arts Entertainment & Recreation'
 'Arts Entertainment & Recreation'
 'Wholesale Trade,Accommodation & Food Services' 'Utilities']


Колонку country нужно убрать, так как она не даёт никакой информации. Категории же можно оставить для дальнейшего анализа

In [11]:
ds = dataset.copy();
ds.drop(columns=['country'], inplace=True);
ds.head(3)


,id,dateAdded,dateUpdated,address,categories,primaryCategories,city,keys,latitude,longitude,name,postalCode,province,reviews.date,reviews.dateSeen,reviews.rating,reviews.sourceURLs,reviews.text,reviews.title,reviews.userCity,reviews.userProvince,reviews.username,sourceURLs,websites
0,AVwc252WIN2L1WUfpqLP,2016-10-30T21:42:42Z,2018-09-10T21:06:27Z,5921 Valencia Cir,"Hotels,Hotels and motels,Hotel and motel reser...",Accommodation & Food Services,Rancho Santa Fe,us/ca/ranchosantafe/5921valenciacir/359754519,32.990959,-117.186136,Rancho Valencia Resort Spa,92067,CA,2013-11-14T00:00:00Z,"2016-08-03T00:00:00Z,2016-07-26T00:00:00Z,2016...",5.0,https://www.hotels.com/hotel/125419/reviews%20/,Our experience at Rancho Valencia was absolute...,Best romantic vacation ever!!!!,NaN,NaN,Paula,http://www.hotels.com/ho125419/%25252525253Flo...,http://www.ranchovalencia.com
1,AVwc252WIN2L1WUfpqLP,2016-10-30T21:42:42Z,2018-09-10T21:06:27Z,5921 Valencia Cir,"Hotels,Hotels and motels,Hotel and motel reser...",Accommodation & Food Services,Rancho Santa Fe,us/ca/ranchosantafe/5921valenciacir/359754519,32.990959,-117.186136,Rancho Valencia Resort Spa,92067,CA,2014-07-06T00:00:00Z,"2016-08-02T00:00:00Z,2016-08-26T00:00:00Z,2016...",5.0,https://www.hotels.com/hotel/125419/reviews%20/,Amazing place. Everyone was extremely warm and...,Sweet sweet serenity,NaN,NaN,D,http://www.hotels.com/ho125419/%25252525253Flo...,http://www.ranchovalencia.com
2,AVwc252WIN2L1WUfpqLP,2016-10-30T21:42:42Z,2018-09-10T21:06:27Z,5921 Valencia Cir,"Hotels,Hotels and motels,Hotel and motel reser...",Accommodation & Food Services,Rancho Santa Fe,us/ca/ranchosantafe/5921valenciacir/359754519,32.990959,-117.186136,Rancho Valencia Resort Spa,92067,CA,2015-01-02T00:00:00Z,"2016-11-15T00:00:00Z,2016-08-23T00:00:00Z,2016...",5.0,https://www.hotels.com/hotel/125419/reviews%20/,We booked a 3 night stay at Rancho Valencia to...,Amazing Property and Experience,NaN,NaN,Ron,http://www.hotels.com/ho125419/%25252525253Flo...,http://www.ranchovalencia.com


##Замена формата

В исходном датасете даты представлены в формате ISO 8601 в формате UTC. Преобразуем в более читаемый вариант и уберём смещение по времени.

Также явно укажем числовые поля.


In [12]:
#change 'dateAdded' format to date
ds["dateAdded"] = ds["dateAdded"].astype("datetime64")
ds['dateAdded'] = [x.strftime('%Y-%m-%d %H:%M:%S') for x in ds.dateAdded];
ds["dateAdded"] = ds["dateAdded"].astype("datetime64")

#change 'dateUpdated' format to date
ds["dateUpdated"] = ds["dateUpdated"].astype("datetime64")
ds['dateUpdated'] = [x.strftime('%Y-%m-%d %H:%M:%S') for x in ds.dateUpdated];
ds["dateUpdated"] = ds["dateUpdated"].astype("datetime64")

#change 'latitude' format to float
ds["latitude"] = ds["latitude"].astype("float")

#change 'longitude' format to float
ds["longitude"] = ds["longitude"].astype("float")

#change 'reviews.date' format to date
ds["reviews.date"] = ds["reviews.date"].astype("datetime64")
ds['reviews.date'] = [x.strftime('%Y-%m-%d %H:%M:%S') for x in ds['reviews.date']];
ds["reviews.date"] = ds["reviews.date"].astype("datetime64")

#change 'reviews.rating' format to float
ds["reviews.rating"] = ds["reviews.rating"].astype("float")

ds.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10000 entries, 0 to 9999
Data columns (total 24 columns):
 #   Column                Non-Null Count  Dtype         
---  ------                --------------  -----         
 0   id                    10000 non-null  object        
 1   dateAdded             10000 non-null  datetime64[ns]
 2   dateUpdated           10000 non-null  datetime64[ns]
 3   address               10000 non-null  object        
 4   categories            10000 non-null  object        
 5   primaryCategories     10000 non-null  object        
 6   city                  10000 non-null  object        
 7   keys                  10000 non-null  object        
 8   latitude              10000 non-null  float64       
 9   longitude             10000 non-null  float64       
 10  name                  10000 non-null  object        
 11  postalCode            10000 non-null  object        
 12  province              10000 non-null  object        
 13  reviews.date     

##Разбиение датасета

Разобьём преобразованные данные на 2 сета:

1) entries - набор, содержащий наибольшее количество данных для наиболее точного анализа. Колонки reviews.userCity и reviews.userProvince содержат много пропусков - удалим.

2) reviews - набор, содержащий меньшее количество данных, но при этом захватывающий колонки reviews.userCity и reviews.userProvince. На его основании можно будет сделать выводы и по ним.

In [13]:
entries = ds.copy();
entries = entries.drop(columns=['reviews.userCity', 'reviews.userProvince'])
entries = entries.dropna(thresh=(len(entries.columns)));
print(entries.info())

print('<--------------------------------------------------------------------->')

print(entries.shape)

<class 'pandas.core.frame.DataFrame'>
Int64Index: 9998 entries, 0 to 9999
Data columns (total 22 columns):
 #   Column              Non-Null Count  Dtype         
---  ------              --------------  -----         
 0   id                  9998 non-null   object        
 1   dateAdded           9998 non-null   datetime64[ns]
 2   dateUpdated         9998 non-null   datetime64[ns]
 3   address             9998 non-null   object        
 4   categories          9998 non-null   object        
 5   primaryCategories   9998 non-null   object        
 6   city                9998 non-null   object        
 7   keys                9998 non-null   object        
 8   latitude            9998 non-null   float64       
 9   longitude           9998 non-null   float64       
 10  name                9998 non-null   object        
 11  postalCode          9998 non-null   object        
 12  province            9998 non-null   object        
 13  reviews.date        9998 non-null   datetime64[n

In [14]:
reviews = ds.copy().dropna(thresh=(len(ds.columns)));
print(reviews.info())

print('<--------------------------------------------------------------------->')

print(reviews.shape)

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2657 entries, 3 to 9999
Data columns (total 24 columns):
 #   Column                Non-Null Count  Dtype         
---  ------                --------------  -----         
 0   id                    2657 non-null   object        
 1   dateAdded             2657 non-null   datetime64[ns]
 2   dateUpdated           2657 non-null   datetime64[ns]
 3   address               2657 non-null   object        
 4   categories            2657 non-null   object        
 5   primaryCategories     2657 non-null   object        
 6   city                  2657 non-null   object        
 7   keys                  2657 non-null   object        
 8   latitude              2657 non-null   float64       
 9   longitude             2657 non-null   float64       
 10  name                  2657 non-null   object        
 11  postalCode            2657 non-null   object        
 12  province              2657 non-null   object        
 13  reviews.date      

#Анализ



##Проверка актуальности данных

Оценим качество и актуальность данных с помощью гистограм на основании дат добавления (dateAdded) и последнего обновления (dateUpdated) соответственно.

In [15]:
datesAdded = entries.copy().drop_duplicates(subset = ["name"]).groupby([entries["dateAdded"].dt.year, entries["dateAdded"].dt.month]).count();
datesAddedY = datesAdded.id.values;
datesAddedX = [(str(i[0]) + ':' + str(i[1])) for i in datesAdded.index.values];

#data_quality = go.Figure(
    #data=[go.Bar(x=datesAddedX, y=datesAddedY)]
#);
data_quality = px.bar(x=datesAddedX, y=datesAddedY, labels={'x':'Дата', 'y':'Количество новых записей'})
data_quality.update_layout(
    margin=dict(l=0, r=30, t=30, b=0),
    title="Гистограмма количества добавленных сайтов за месяц",
    xaxis_title="Время",
    yaxis_title="Новые записи",
    font_family="Courier New",
    font_color="blue",
    title_font_family="Times New Roman",
    title_font_color="green"
    )

data_quality.show();

In [16]:
datesUpdated = entries.copy().drop_duplicates(subset = ["name"]).groupby([entries["dateUpdated"].dt.year, entries["dateUpdated"].dt.month]).count();
datesUpdatedY = datesUpdated.id.values;
datesUpdatedX = [(str(i[0]) + ':' + str(i[1])) for i in datesUpdated.index.values];

#data_relevance = go.Figure(
    #data=[go.Bar(x=datesAddedX, y=datesAddedY)]
#);
data_relevance = px.bar(x=datesUpdatedX, y=datesUpdatedY, labels={'x':'Дата', 'y':'Количество обновлённых записей'})
data_relevance.update_layout(
    margin=dict(l=0, r=30, t=30, b=0),
    title="Гистограмма количества обновлённых сайтов за месяц",
    xaxis_title="Время",
    yaxis_title="Обновлённые записи записи",
    font_family="Courier New",
    font_color="blue",
    title_font_family="Times New Roman",
    title_font_color="green"
    )

data_relevance.show();

Как мы можем заметить, многие компании наблюдаются уже не первый год, что хорошо

В целом видно, что информация обновляется достаточно часто. Но датасет не самый свежий (соревнование давно закончилось), поэтому скажем, что его стоило бы обновить для более точного анализа

##Анализ компаний по главным категориям

Посмотрим, какие категории у нас имеются и в каком соотношении они представляют датасет

In [17]:
primaryCategories = entries.copy().drop_duplicates(subset = ["name"]).groupby([entries["primaryCategories"]]).count();
primaryCategoriesValues = primaryCategories.id.values;
primaryCategoriesNames = primaryCategories.index;

data_primary_categories = go.Figure(
    data=[go.Pie(labels=primaryCategoriesNames, values=primaryCategoriesValues, hole=.3)]
    );
data_primary_categories.update_layout(
    margin=dict(l=0, r=30, t=30, b=0),
    title="Диаграмма главных категорий компаний",
    title_font_family="Times New Roman",
    title_font_color="green"
    )

data_primary_categories.show();

Как мы можем увидеть, практически весь датасет представляет собой информацию об компаниях по проживанию и питанию, то есть из отелей. Так как нас интересует именно эта информация, отфильтруем наши датасеты по главной категории.


In [18]:
entries = entries[entries.primaryCategories=='Accommodation & Food Services']
reviews = reviews[reviews.primaryCategories=='Accommodation & Food Services']

## Представление распределения отелей на карте

Рассмотрим распределение отелей на карте

In [19]:
hotelsCount = entries.copy().drop_duplicates(subset = ["name"]).groupby([entries["city"]]).count();
hotelsAvg = entries.copy().drop_duplicates(subset = ["name"]).groupby([entries["city"]]).mean();
hotels_map_dist = go.Figure();

hotelsCount['text'] = hotelsCount.index.astype('str') + '<br>Отелей: ' + hotelsCount['id'].astype(str)

hotels_map_dist = go.Figure(go.Scattermapbox(lat=hotelsAvg['latitude'], 
                                 lon=hotelsAvg['longitude'], 
                                 text=hotelsCount['text'],
                                 marker=dict(colorbar=dict(title="Количество отелей"),
                                             color=hotelsCount['id'],
                                             size=hotelsCount['id'])))
map_center = go.layout.mapbox.Center(lat=(hotelsAvg['latitude'].max()+hotelsAvg['latitude'].min())/2, 
                                     lon=(hotelsAvg['longitude'].max()+hotelsAvg['longitude'].min())/2)
hotels_map_dist.update_layout(mapbox_style="open-street-map",
                  mapbox=dict(center=map_center, zoom=2.5))
hotels_map_dist.show()

#hotels_map_dist.show()

Можно сказать, что в центре материка отелей значительно меньше. Основное количество сосредоточено в больших городах и на побережье.

##Средние отзывы отелей по штатам

Рассмотрим, какие штаты являются наиболее перспективными для размещения отеля, по отзывам клиентов

In [20]:
reviewsInStates = entries.copy().drop_duplicates(subset = ["name"]).groupby([entries["province"]]).mean().sort_values('reviews.rating');
reviewsInStates = reviewsInStates.iloc[::-1]
reviewsInStatesY = reviewsInStates['reviews.rating'].values;
reviewsInStatesX = reviewsInStates.index;

#data_quality = go.Figure(
    #data=[go.Bar(x=datesAddedX, y=datesAddedY)]
#);
reviews_in_states_gist = px.bar(x=reviewsInStatesX, y=reviewsInStatesY, labels={'x':'Штаты', 'y':'Средний рейтинг отелей'})
reviews_in_states_gist.update_layout(
    margin=dict(l=0, r=30, t=30, b=0),
    title="Гистограмма среднего рейтинга отелей по штатам",
    xaxis_title="Штаты",
    yaxis_title="Средняя оценка",
    font_family="Courier New",
    font_color="blue",
    title_font_family="Times New Roman",
    title_font_color="green"
    )

reviews_in_states_gist.show();

In [21]:
hotelsNumInStates = entries.copy().drop_duplicates(subset = ["name"]).groupby([entries["province"]]).count().sort_values('id');
hotelsNumInStates = hotelsNumInStates.iloc[::-1]
hotelsNumInStatesY = hotelsNumInStates['id'].values;
hotelsNumInStatesX = hotelsNumInStates.index;

#data_quality = go.Figure(
    #data=[go.Bar(x=datesAddedX, y=datesAddedY)]
#);
hotels_num_in_states = px.bar(x=hotelsNumInStatesX, y=hotelsNumInStatesY, labels={'x':'Штаты', 'y':'Количество отелей'})
hotels_num_in_states.update_layout(
    margin=dict(l=0, r=30, t=30, b=0),
    title="Гистограмма количества отелей в штатах",
    xaxis_title="Штаты",
    yaxis_title="Количество отелей",
    font_family="Courier New",
    font_color="blue",
    title_font_family="Times New Roman",
    title_font_color="green"
    )

hotels_num_in_states.show();

Как мы можем заметить, штатами с наиболее хорошими отелями являются:

* RI (Rhode Island) - 5.0

* WY (Wyoming) - 4.42

* SD (South Dakota) - 4.37

Но нельзя сказать, что в этих штатах уже расположены наилучшие отели и конкуренция бесполезна. Так в топ-1 штате находится всего лишь один отель.
Но мы же хотим оценить и дать какой-нибудь конкретный вывод нашим клиентам!

Поэтому построим простую линейную регрессионную модель, которая будет зависима от двух критериев (средний отзыв по штату и число отелей в штате). Обучена же модель будет на заранее подготовленных значениях, которые мы сами субъективно сгенерируем в голове.

In [22]:
training_points = np.array([
                    [1, 1], [1, 0.5], [1, 0.1],
                    [0.6, 1], [0.6, 0.5], [0.6, 0.1],
                    [0.2, 1], [0.2, 0.5], [0.2, 0.1]
                     ]);
training_values = np.array([0, 0.3, 0.7, 0.4, 0.5, 0.6, 0.6, 0.8, 1])

linear_regression_model = LinearRegression()
linear_regression_model.fit(training_points, training_values)

mean_rate_data = reviewsInStates = entries.copy().drop_duplicates(subset = ["name"]).groupby([entries["province"]]).mean();
mean_rate_data = mean_rate_data['reviews.rating'] / mean_rate_data['reviews.rating'].max();
num_hotels_data = entries.copy().drop_duplicates(subset = ["name"]).groupby([entries["province"]]).count();
num_hotels_data =  num_hotels_data['id'] / num_hotels_data['id'].max();

result = pd.DataFrame({'Mean_rating' : mean_rate_data.values,
                       'Hotels_num': num_hotels_data.values
                       });

result.index = mean_rate_data.index;
result['evaluation'] = linear_regression_model.predict(np.vstack((result['Mean_rating'], result['Hotels_num'])).T);

In [23]:
evaluateStateData = result.copy().sort_values('evaluation');
evaluateStateData = evaluateStateData.iloc[::-1]
evaluateStateDataY = evaluateStateData['evaluation'].values;
evaluateStateDataX = evaluateStateData.index;

state_evaluation = px.bar(x = evaluateStateDataX, y = evaluateStateDataY, labels={'x':'Штаты', 'y':'Количество отелей'})
state_evaluation.update_layout(
    margin=dict(l=0, r=30, t=30, b=0),
    title="Гистограмма количества отелей в штатах",
    xaxis_title="Штаты",
    yaxis_title="Количество отелей",
    font_family="Courier New",
    font_color="blue",
    title_font_family="Times New Roman",
    title_font_color="green"
    )

state_evaluation.show();

Таким образом, по нашему анализу наилучшими штатами для открытия своего бизнеса, судя по средним отзывам и количеству отелей, являются:


In [24]:
print(result['evaluation'].head(10));

province
AK    0.733500
AR    0.615695
AZ    0.571490
CA    0.227648
CO    0.585102
CT    0.644146
DE    0.702479
FL    0.326605
GA    0.560828
HI    0.639102
Name: evaluation, dtype: float64


## Выявление связи между населением штата и количеством отзывов об отелях

Рассмотрим, влияет ли населённость штата на количество суммарных отзывов об отелях.

В нашем исходном датасете нет информации об населении штата, поэтому мы подключим дополнительный.

###Получение и предобработка информации о населённости штатов

####Получение данных

In [25]:
df = pd.read_excel("/content/drive/MyDrive/Reviews_Analysis/USA_Demography.xlsx", skipfooter=5,header=1)
df.columns=map(lambda x: str(x),df.columns)
df

,Area,Census,Estimates Base,2010,2011,2012,2013,2014,2015,2016,2017,2018,2019
0,"Abbeville city, Alabama",2688,2705,2699,2694,2643,2628,2608,2600,2584,2575,2571,2560
1,"Adamsville city, Alabama",4522,4506,4500,4493,4471,4449,4420,4390,4356,4327,4308,4281
2,"Addison town, Alabama",758,754,751,750,743,742,739,734,731,726,723,718
3,"Akron town, Alabama",356,356,355,347,347,343,338,339,333,332,331,328
4,"Alabaster city, Alabama",30352,31112,31209,31375,31684,31980,32182,32772,33017,33275,33413,33487
...,...,...,...,...,...,...,...,...,...,...,...,...,...
19492,"Thayne town, Wyoming",366,366,365,364,359,367,371,371,376,383,383,394
19493,"Thermopolis town, Wyoming",3009,3016,3015,3012,3026,3027,2999,2955,2914,2936,2861,2764
19494,"Torrington city, Wyoming",6501,6501,6638,6722,6761,6736,6719,6763,6652,6692,6663,6624
19495,"Upton town, Wyoming",1100,1100,1098,1089,1081,1088,1095,1104,1105,1062,1057,1056


####Предобработка данных

Видим, что получаем неудобный первый столбец, поэтому разделим их на два столбца.

In [26]:
def function(x):
  return x.split(', ')[-1]
df['State'] = df['Area'].apply(function)
df

,Area,Census,Estimates Base,2010,2011,2012,2013,2014,2015,2016,2017,2018,2019,State
0,"Abbeville city, Alabama",2688,2705,2699,2694,2643,2628,2608,2600,2584,2575,2571,2560,Alabama
1,"Adamsville city, Alabama",4522,4506,4500,4493,4471,4449,4420,4390,4356,4327,4308,4281,Alabama
2,"Addison town, Alabama",758,754,751,750,743,742,739,734,731,726,723,718,Alabama
3,"Akron town, Alabama",356,356,355,347,347,343,338,339,333,332,331,328,Alabama
4,"Alabaster city, Alabama",30352,31112,31209,31375,31684,31980,32182,32772,33017,33275,33413,33487,Alabama
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
19492,"Thayne town, Wyoming",366,366,365,364,359,367,371,371,376,383,383,394,Wyoming
19493,"Thermopolis town, Wyoming",3009,3016,3015,3012,3026,3027,2999,2955,2914,2936,2861,2764,Wyoming
19494,"Torrington city, Wyoming",6501,6501,6638,6722,6761,6736,6719,6763,6652,6692,6663,6624,Wyoming
19495,"Upton town, Wyoming",1100,1100,1098,1089,1081,1088,1095,1104,1105,1062,1057,1056,Wyoming


Штаты в первом датасете имеются сокращенные названия, пэотому сделаем Dictionary.


In [27]:
def funct(x):
  x=x.split(', ')[-1]
  dict = {
        'Alaska': 'AK',
        'Alabama': 'AL',
        'Arkansas': 'AR',
        'Arizona': 'AZ',
        'California': 'CA',
        'Colorado': 'CO',
        'Connecticut': 'CT',
        'Delaware': 'DE',
        'Florida': 'FL',
        'Georgia': 'GA',
        'Hawaii': 'HI',
        'Iowa': 'IA',
        'Idaho': 'ID',
        'Illinois': 'IL',
        'Indiana': 'IN',
        'Kansas': 'KS',
        'Kentucky': 'KY',
        'Louisiana': 'LA',
        'Massachusetts': 'MA',
        'Maryland': 'MD',
        'Maine': 'ME',
        'Michigan': 'MI',
        'Minnesota': 'MN',
        'Missouri': 'MO',
        'Mississippi': 'MS',
        'Montana': 'MT',
        'North Carolina': 'NC',
        'North Dakota': 'ND',
        'Nebraska': 'NE',
        'New Hampshire': 'NH',
        'New Jersey': 'NJ',
        'New Mexico': 'NM',
        'Nevada': 'NV',
        'New York': 'NY',
        'Ohio': 'OH',
        'Oklahoma': 'OK',
        'Oregon': 'OR',
        'Pennsylvania': 'PA',
        'Rhode Island': 'RI',
        'South Carolina': 'SC',
        'South Dakota': 'SD',
        'Tennessee': 'TN',
        'Texas': 'TX',
        'Utah': 'UT',
        'Virginia': 'VA',
        'Vermont': 'VT',
        'Washington': 'WA',
        'Wisconsin': 'WI',
        'West Virginia': 'WV',
        'Wyoming': 'WY',
        'District of Columbia': 'DC'
    }
  return dict.get(x)
    
df['province'] = df['State'].apply(funct)
df

,Area,Census,Estimates Base,2010,2011,2012,2013,2014,2015,2016,2017,2018,2019,State,province
0,"Abbeville city, Alabama",2688,2705,2699,2694,2643,2628,2608,2600,2584,2575,2571,2560,Alabama,AL
1,"Adamsville city, Alabama",4522,4506,4500,4493,4471,4449,4420,4390,4356,4327,4308,4281,Alabama,AL
2,"Addison town, Alabama",758,754,751,750,743,742,739,734,731,726,723,718,Alabama,AL
3,"Akron town, Alabama",356,356,355,347,347,343,338,339,333,332,331,328,Alabama,AL
4,"Alabaster city, Alabama",30352,31112,31209,31375,31684,31980,32182,32772,33017,33275,33413,33487,Alabama,AL
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
19492,"Thayne town, Wyoming",366,366,365,364,359,367,371,371,376,383,383,394,Wyoming,WY
19493,"Thermopolis town, Wyoming",3009,3016,3015,3012,3026,3027,2999,2955,2914,2936,2861,2764,Wyoming,WY
19494,"Torrington city, Wyoming",6501,6501,6638,6722,6761,6736,6719,6763,6652,6692,6663,6624,Wyoming,WY
19495,"Upton town, Wyoming",1100,1100,1098,1089,1081,1088,1095,1104,1105,1062,1057,1056,Wyoming,WY


Получим кол-во жителей по штатам с сокращенными названиями.

In [28]:
population = df.groupby(by='province')['2018'].sum()
population

province
AK      500653
AL     2988163
AR     1991649
AZ     5691895
CA    32919180
CO     4222032
CT     1382875
DC      701547
DE      273786
FL    10680862
GA     4629537
HI      346945
IA     2517283
ID     1229130
IL    11081594
IN     4476548
KS     2416937
KY     2469316
LA     2188619
MA     3636544
MD     1535928
ME      375864
MI     5063690
MN     4635029
MO     4055273
MS     1511574
MT      576629
NC     5891694
ND      589048
NE     1498193
NH      426321
NJ     4236064
NM     1397868
NV     1717570
NY    12464918
OH     7596643
OK     3028855
OR     2936852
PA     5595890
RI      547027
SC     1840821
SD      620665
TN     4064272
TX    21103807
UT     2949706
VA     3085272
VT      150164
WA     4909661
WI     4189381
WV      613203
WY      386357
Name: 2018, dtype: int64

###Анализ задачи

In [29]:
reviews_by_states = entries.copy().groupby([entries["province"]]).count().id;

population_map = population.to_dict();

population_column = [population_map[i] for i in reviews_by_states.index.array];

population_and_reviews = pd.DataFrame({'reviews_amount': reviews_by_states.values, 'population': population_column})
population_and_reviews.index = reviews_by_states.index;
population_and_reviews

,reviews_amount,population
province,,
AK,21,500653
AR,33,1991649
AZ,212,5691895
CA,1036,32919180
CO,134,4222032
CT,30,1382875
DE,56,273786
FL,854,10680862
GA,281,4629537


Рассмотрим, коррелируют ли данные показатели


In [30]:
population_and_reviews.corr().style.format("{:.2}").background_gradient(cmap='coolwarm', axis=1)

,reviews_amount,population
reviews_amount,1.0,0.53
population,0.53,1.0


В целом можно сказать, что между этими величинами существует некая связь. Не очень сильная, не очень слабая...

Изобразим на графике данные для наглядности



In [31]:
reviewsPopulationPlotY = population_and_reviews.copy().sort_values('population')['reviews_amount'].values;
reviewsPopulationPlotX = population_and_reviews.sort_values('population')['population'].values;

reviews_population_plot = px.line(x = reviewsPopulationPlotX, y = reviewsPopulationPlotY, labels={'x':'Население', 'y':'Количество отзывов'}, text=population_and_reviews.index)
reviews_population_plot.update_layout(
    margin=dict(l=0, r=30, t=30, b=0),
    title="Зависимость количества отзывов в штате от населения",
    xaxis_title="Население",
    yaxis_title="Количество отзывов",
    font_family="Courier New",
    font_color="blue",
    title_font_family="Times New Roman",
    title_font_color="green"
    )

reviews_population_plot.show();

Рассмотрим, влияет ли количество отелей на количество отзывов (что звучит логичнее, на наш субъективный взгляд)


In [32]:
hotels_num_and_reviews = pd.DataFrame({'reviews_amount': reviews_by_states.values, 'hotels amount': entries.copy().drop_duplicates(subset = ["name"]).groupby([entries["province"]]).count()['id'].values})
hotels_num_and_reviews.index = reviews_by_states.index;
hotels_num_and_reviews

,reviews_amount,hotels amount
province,,
AK,21,9
AR,33,18
AZ,212,39
CA,1036,159
CO,134,41
CT,30,8
DE,56,8
FL,854,118
GA,281,53


In [33]:
hotels_num_and_reviews.corr().style.format("{:.2}").background_gradient(cmap='coolwarm', axis=1)

,reviews_amount,hotels amount
reviews_amount,1.0,0.77
hotels amount,0.77,1.0


Как мы можем заметить, коэффициент корреляции значительно выше, и наше предположение действительно верно

In [34]:
reviewsHotelsNumPlotY = hotels_num_and_reviews.copy().sort_values('hotels amount')['reviews_amount'].values;
reviewsHotelsNumPlotX = hotels_num_and_reviews.sort_values('hotels amount')['hotels amount'].values;

reviews_hotel_num_plot = px.line(x = reviewsHotelsNumPlotX, y = reviewsHotelsNumPlotY, labels={'x':'Количество отелей', 'y':'Количество отзывов'}, text=population_and_reviews.index)
reviews_hotel_num_plot.update_layout(
    margin=dict(l=0, r=30, t=30, b=0),
    title="Зависимость количества отзывов в штате от количества отелей",
    xaxis_title="Количество отелей",
    yaxis_title="Количество отзывов",
    font_family="Courier New",
    font_color="blue",
    title_font_family="Times New Roman",
    title_font_color="green"
    )

reviews_hotel_num_plot.show();

Таким образом, количество населения штата имеет влияение на суммарное количество отзывов по штату.

Но гораздо большее влияние оказывает количество отелей

##Выявление связи между туристическим бюджетом штата и количеством отзывов об отелях

Рассмотрим, влияет ли туристический бюджет штата на количество суммарных отзывов об отелях.

В нашем исходном датасете нет информации об населении штата, поэтому мы также подключим дополнительный.

### Подключение и предобработка информации о туристическом бюджетах штатов

####Получение данных

In [35]:
df2 = pd.read_excel("/content/drive/MyDrive/Reviews_Analysis/2018_ASFIN_State_Totals.xlsx")
df2

,(Thousands of Dollars),United States,Alabama,Alaska,Arizona,Arkansas,California,Colorado,Connecticut,Delaware,Florida,Georgia,Hawaii,Idaho,Illinois,Indiana,Iowa,Kansas,Kentucky,Louisiana,Maine,Maryland,Massachusetts,Michigan,Minnesota,Mississippi,Missouri,Montana,Nebraska,Nevada,New Hampshire,New Jersey,New Mexico,New York,North Carolina,North Dakota,Ohio,Oklahoma,Oregon,Pennsylvania,Rhode Island,South Carolina,South Dakota,Tennessee,Texas,Utah,Vermont,Virginia,Washington,West Virginia,Wisconsin,Wyoming
0,Total revenue,2.707921e+09,34635220.0,10304503.0,48210938.0,26378654.0,430760445.0,43736624.0,37093455.0,10721781.0,110898792.0,58156055.0,16535025.0,12068171.0,100374663.0,46276655.0,29099642.0,22465649.0,36322780.0,38443907.0,11187326.0,50072463.0,71117614.0,81644738.0,58515701.0,23839443.0,40828091.0,11189063.0,12446993.0,22026408.0,9741498.0,85061613.0,21164220.0,228736608.0,69139215.0,8965071.0,103319265.0,27516131.0,44374588.0,110623233.0,10263247.0,36214436.0,5652171.0,36440858.0,175350481.0,26274300.0,7159827.0,57339098.0,70873183.0,15856198.0,54852031.0,7652548.0
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,General revenue,2.100789e+09,28259466.0,8166711.0,38334391.0,21600454.0,321329882.0,31001206.0,30276609.0,9275665.0,91783197.0,45324067.0,13823391.0,9158102.0,72721191.0,40546768.0,24071455.0,19078076.0,29299781.0,29170664.0,8815208.0,42092006.0,58177867.0,65660224.0,44508071.0,19137029.0,30927033.0,7031628.0,10470221.0,16087532.0,7566526.0,66137838.0,17494796.0,175752400.0,56437728.0,7746544.0,68780839.0,21994150.0,32133520.0,87715628.0,8397297.0,28158714.0,4400980.0,30358779.0,138316877.0,20027791.0,6339130.0,50877881.0,50965346.0,13174928.0,36281374.0,5602117.0
3,Intergovernmental revenue,6.873869e+08,10303451.0,3457365.0,16127713.0,7877352.0,102060783.0,9402733.0,7440296.0,2619277.0,28298587.0,15284021.0,3013717.0,2819674.0,22204179.0,15075489.0,6075843.0,4072601.0,11970566.0,13648190.0,3012515.0,12923538.0,17184464.0,21365685.0,11773755.0,8271815.0,11957330.0,3124693.0,3264232.0,5454068.0,2799206.0,18094137.0,7534520.0,64587516.0,18700690.0,1742138.0,24388743.0,7100962.0,10470949.0,29418893.0,2883313.0,9916827.0,1504052.0,11950090.0,47201933.0,4695609.0,2091212.0,10957750.0,14446364.0,4959925.0,9494180.0,2363993.0
4,Taxes,1.027085e+09,11055577.0,1656352.0,16293917.0,9819284.0,178437038.0,14924842.0,18934012.0,4219706.0,45961204.0,23602510.0,7714451.0,4848359.0,39857069.0,19397879.0,10088480.0,9546790.0,12059970.0,11357686.0,4410632.0,22427037.0,29654803.0,30046808.0,26697469.0,7890571.0,13025070.0,3003980.0,5393093.0,9157036.0,2920888.0,35365046.0,5539329.0,88541099.0,27855861.0,4205184.0,29068270.0,9429242.0,12640306.0,40709545.0,3491697.0,10530212.0,1917548.0,14269061.0,60328843.0,9414073.0,3284231.0,23484945.0,26579324.0,5442628.0,18748320.0,1837401.0
5,General sales,3.197024e+08,2786833.0,NaN,7687992.0,3498073.0,39682734.0,3211909.0,4400808.0,NaN,29562900.0,5938448.0,3529065.0,1790830.0,11336866.0,7795091.0,3279789.0,3304091.0,3600598.0,4252693.0,1529113.0,4716179.0,6490305.0,9595949.0,5830256.0,3557752.0,3686274.0,NaN,1900037.0,5095689.0,NaN,10459419.0,2073118.0,14820163.0,8009850.0,912532.0,12148485.0,2855176.0,NaN,10920832.0,1048957.0,3303220.0,1103624.0,7469547.0,36129876.0,2784489.0,397691.0,4076636.0,15643017.0,1311930.0,5484375.0,689207.0
6,Selective sales,1.646055e+08,2747974.0,252970.0,2020702.0,1336910.0,17662374.0,2500334.0,3196187.0,585645.0,8893715.0,3194517.0,1202967.0,605287.0,7362911.0,4324467.0,1407284.0,1190718.0,2113829.0,2612030.0,721180.0,4851065.0,2532057.0,4802420.0,4784552.0,1473102.0,1852953.0,655034.0,601983.0,2233767.0,1025902.0,4965088.0,801689.0,11579156.0,4203608.0,482971.0,5908503.0,1339096.0,1654829.0,10113420.0,633064.0,1644958.0,481465.0,2873537.0,15332333.0,1116797.0,697630.0,3006106.0,4684503.0,1419809.0,

####Предобработка данных

Посмотрим, какие колонки содержатся в данном датасете

In [36]:
df2.columns

Index(['(Thousands of Dollars)', 'United States', 'Alabama', 'Alaska',
       'Arizona', 'Arkansas', 'California', 'Colorado', 'Connecticut',
       'Delaware', 'Florida', 'Georgia', 'Hawaii', 'Idaho', 'Illinois',
       'Indiana', 'Iowa', 'Kansas', 'Kentucky', 'Louisiana', 'Maine',
       'Maryland', 'Massachusetts', 'Michigan', 'Minnesota', 'Mississippi',
       'Missouri', 'Montana', 'Nebraska', 'Nevada', 'New Hampshire',
       'New Jersey', 'New Mexico', 'New York', 'North Carolina',
       'North Dakota', 'Ohio', 'Oklahoma', 'Oregon', 'Pennsylvania',
       'Rhode Island', 'South Carolina', 'South Dakota', 'Tennessee', 'Texas',
       'Utah', 'Vermont', 'Virginia', 'Washington', 'West Virginia',
       'Wisconsin', 'Wyoming'],
      dtype='object')

Как мы можем увидеть, мы имеем следующую информацию:

Каждая строчка представляет конкретный пункт бюджета (первая колонка), суммарный бюджет по Америке (вторая колонка) и бюджеты по штатам (все оставшиеся)

Мы работаем с сокращёнными названиями штатов в исходном датасете, поэтому приведём их к такому виду и здесь

In [37]:
df2.rename(columns={
        'Alaska': 'AK',
        'Alabama': 'AL',
        'Arkansas': 'AR',
        'Arizona': 'AZ',
        'California': 'CA',
        'Colorado': 'CO',
        'Connecticut': 'CT',
        'Delaware': 'DE',
        'Florida': 'FL',
        'Georgia': 'GA',
        'Hawaii': 'HI',
        'Iowa': 'IA',
        'Idaho': 'ID',
        'Illinois': 'IL',
        'Indiana': 'IN',
        'Kansas': 'KS',
        'Kentucky': 'KY',
        'Louisiana': 'LA',
        'Massachusetts': 'MA',
        'Maryland': 'MD',
        'Maine': 'ME',
        'Michigan': 'MI',
        'Minnesota': 'MN',
        'Missouri': 'MO',
        'Mississippi': 'MS',
        'Montana': 'MT',
        'North Carolina': 'NC',
        'North Dakota': 'ND',
        'Nebraska': 'NE',
        'New Hampshire': 'NH',
        'New Jersey': 'NJ',
        'New Mexico': 'NM',
        'Nevada': 'NV',
        'New York': 'NY',
        'Ohio': 'OH',
        'Oklahoma': 'OK',
        'Oregon': 'OR',
        'Pennsylvania': 'PA',
        'Rhode Island': 'RI',
        'South Carolina': 'SC',
        'South Dakota': 'SD',
        'Tennessee': 'TN',
        'Texas': 'TX',
        'Utah': 'UT',
        'Virginia': 'VA',
        'Vermont': 'VT',
        'Washington': 'WA',
        'Wisconsin': 'WI',
        'West Virginia': 'WV',
        'Wyoming': 'WY',
        'District of Columbia': 'DC'
    }, inplace=True)
df2

,(Thousands of Dollars),United States,AL,AK,AZ,AR,CA,CO,CT,DE,FL,GA,HI,ID,IL,IN,IA,KS,KY,LA,ME,MD,MA,MI,MN,MS,MO,MT,NE,NV,NH,NJ,NM,NY,NC,ND,OH,OK,OR,PA,RI,SC,SD,TN,TX,UT,VT,VA,WA,WV,WI,WY
0,Total revenue,2.707921e+09,34635220.0,10304503.0,48210938.0,26378654.0,430760445.0,43736624.0,37093455.0,10721781.0,110898792.0,58156055.0,16535025.0,12068171.0,100374663.0,46276655.0,29099642.0,22465649.0,36322780.0,38443907.0,11187326.0,50072463.0,71117614.0,81644738.0,58515701.0,23839443.0,40828091.0,11189063.0,12446993.0,22026408.0,9741498.0,85061613.0,21164220.0,228736608.0,69139215.0,8965071.0,103319265.0,27516131.0,44374588.0,110623233.0,10263247.0,36214436.0,5652171.0,36440858.0,175350481.0,26274300.0,7159827.0,57339098.0,70873183.0,15856198.0,54852031.0,7652548.0
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,General revenue,2.100789e+09,28259466.0,8166711.0,38334391.0,21600454.0,321329882.0,31001206.0,30276609.0,9275665.0,91783197.0,45324067.0,13823391.0,9158102.0,72721191.0,40546768.0,24071455.0,19078076.0,29299781.0,29170664.0,8815208.0,42092006.0,58177867.0,65660224.0,44508071.0,19137029.0,30927033.0,7031628.0,10470221.0,16087532.0,7566526.0,66137838.0,17494796.0,175752400.0,56437728.0,7746544.0,68780839.0,21994150.0,32133520.0,87715628.0,8397297.0,28158714.0,4400980.0,30358779.0,138316877.0,20027791.0,6339130.0,50877881.0,50965346.0,13174928.0,36281374.0,5602117.0
3,Intergovernmental revenue,6.873869e+08,10303451.0,3457365.0,16127713.0,7877352.0,102060783.0,9402733.0,7440296.0,2619277.0,28298587.0,15284021.0,3013717.0,2819674.0,22204179.0,15075489.0,6075843.0,4072601.0,11970566.0,13648190.0,3012515.0,12923538.0,17184464.0,21365685.0,11773755.0,8271815.0,11957330.0,3124693.0,3264232.0,5454068.0,2799206.0,18094137.0,7534520.0,64587516.0,18700690.0,1742138.0,24388743.0,7100962.0,10470949.0,29418893.0,2883313.0,9916827.0,1504052.0,11950090.0,47201933.0,4695609.0,2091212.0,10957750.0,14446364.0,4959925.0,9494180.0,2363993.0
4,Taxes,1.027085e+09,11055577.0,1656352.0,16293917.0,9819284.0,178437038.0,14924842.0,18934012.0,4219706.0,45961204.0,23602510.0,7714451.0,4848359.0,39857069.0,19397879.0,10088480.0,9546790.0,12059970.0,11357686.0,4410632.0,22427037.0,29654803.0,30046808.0,26697469.0,7890571.0,13025070.0,3003980.0,5393093.0,9157036.0,2920888.0,35365046.0,5539329.0,88541099.0,27855861.0,4205184.0,29068270.0,9429242.0,12640306.0,40709545.0,3491697.0,10530212.0,1917548.0,14269061.0,60328843.0,9414073.0,3284231.0,23484945.0,26579324.0,5442628.0,18748320.0,1837401.0
5,General sales,3.197024e+08,2786833.0,NaN,7687992.0,3498073.0,39682734.0,3211909.0,4400808.0,NaN,29562900.0,5938448.0,3529065.0,1790830.0,11336866.0,7795091.0,3279789.0,3304091.0,3600598.0,4252693.0,1529113.0,4716179.0,6490305.0,9595949.0,5830256.0,3557752.0,3686274.0,NaN,1900037.0,5095689.0,NaN,10459419.0,2073118.0,14820163.0,8009850.0,912532.0,12148485.0,2855176.0,NaN,10920832.0,1048957.0,3303220.0,1103624.0,7469547.0,36129876.0,2784489.0,397691.0,4076636.0,15643017.0,1311930.0,5484375.0,689207.0
6,Selective sales,1.646055e+08,2747974.0,252970.0,2020702.0,1336910.0,17662374.0,2500334.0,3196187.0,585645.0,8893715.0,3194517.0,1202967.0,605287.0,7362911.0,4324467.0,1407284.0,1190718.0,2113829.0,2612030.0,721180.0,4851065.0,2532057.0,4802420.0,4784552.0,1473102.0,1852953.0,655034.0,601983.0,2233767.0,1025902.0,4965088.0,801689.0,11579156.0,4203608.0,482971.0,5908503.0,1339096.0,1654829.0,10113420.0,633064.0,1644958.0,481465.0,2873537.0,15332333.0,1116797.0,697630.0,3006106.0,4684503.0,1419809.0,2748521.0,173590.0
7,License taxes,5.715884e+07,521990.0,153636.0,495034.0,409856.0,10504521.0,816832.0,431196.0,1574576.0,2123462.0,719422.0,275600.0,367178.0,2751479.0,749114.0,954503.0,433229.0,499502.0,393129.0,281155.0,850913.0,1154719.0,1945619.0,1476228.0,492415.0,596167.0,379198.0,196215.0,661240.0,435544.0,15422

Первые 2 колонки не имеют для нас ценности - нас интересуют только бюджеты по штатам. Поэтому создадим новый датасет, отбросить лишнее

In [38]:
df3=df2[df2.columns[2:52]]
df3

,AL,AK,AZ,AR,CA,CO,CT,DE,FL,GA,HI,ID,IL,IN,IA,KS,KY,LA,ME,MD,MA,MI,MN,MS,MO,MT,NE,NV,NH,NJ,NM,NY,NC,ND,OH,OK,OR,PA,RI,SC,SD,TN,TX,UT,VT,VA,WA,WV,WI,WY
0,34635220.0,10304503.0,48210938.0,26378654.0,430760445.0,43736624.0,37093455.0,10721781.0,110898792.0,58156055.0,16535025.0,12068171.0,100374663.0,46276655.0,29099642.0,22465649.0,36322780.0,38443907.0,11187326.0,50072463.0,71117614.0,81644738.0,58515701.0,23839443.0,40828091.0,11189063.0,12446993.0,22026408.0,9741498.0,85061613.0,21164220.0,228736608.0,69139215.0,8965071.0,103319265.0,27516131.0,44374588.0,110623233.0,10263247.0,36214436.0,5652171.0,36440858.0,175350481.0,26274300.0,7159827.0,57339098.0,70873183.0,15856198.0,54852031.0,7652548.0
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,28259466.0,8166711.0,38334391.0,21600454.0,321329882.0,31001206.0,30276609.0,9275665.0,91783197.0,45324067.0,13823391.0,9158102.0,72721191.0,40546768.0,24071455.0,19078076.0,29299781.0,29170664.0,8815208.0,42092006.0,58177867.0,65660224.0,44508071.0,19137029.0,30927033.0,7031628.0,10470221.0,16087532.0,7566526.0,66137838.0,17494796.0,175752400.0,56437728.0,7746544.0,68780839.0,21994150.0,32133520.0,87715628.0,8397297.0,28158714.0,4400980.0,30358779.0,138316877.0,20027791.0,6339130.0,50877881.0,50965346.0,13174928.0,36281374.0,5602117.0
3,10303451.0,3457365.0,16127713.0,7877352.0,102060783.0,9402733.0,7440296.0,2619277.0,28298587.0,15284021.0,3013717.0,2819674.0,22204179.0,15075489.0,6075843.0,4072601.0,11970566.0,13648190.0,3012515.0,12923538.0,17184464.0,21365685.0,11773755.0,8271815.0,11957330.0,3124693.0,3264232.0,5454068.0,2799206.0,18094137.0,7534520.0,64587516.0,18700690.0,1742138.0,24388743.0,7100962.0,10470949.0,29418893.0,2883313.0,9916827.0,1504052.0,11950090.0,47201933.0,4695609.0,2091212.0,10957750.0,14446364.0,4959925.0,9494180.0,2363993.0
4,11055577.0,1656352.0,16293917.0,9819284.0,178437038.0,14924842.0,18934012.0,4219706.0,45961204.0,23602510.0,7714451.0,4848359.0,39857069.0,19397879.0,10088480.0,9546790.0,12059970.0,11357686.0,4410632.0,22427037.0,29654803.0,30046808.0,26697469.0,7890571.0,13025070.0,3003980.0,5393093.0,9157036.0,2920888.0,35365046.0,5539329.0,88541099.0,27855861.0,4205184.0,29068270.0,9429242.0,12640306.0,40709545.0,3491697.0,10530212.0,1917548.0,14269061.0,60328843.0,9414073.0,3284231.0,23484945.0,26579324.0,5442628.0,18748320.0,1837401.0
5,2786833.0,NaN,7687992.0,3498073.0,39682734.0,3211909.0,4400808.0,NaN,29562900.0,5938448.0,3529065.0,1790830.0,11336866.0,7795091.0,3279789.0,3304091.0,3600598.0,4252693.0,1529113.0,4716179.0,6490305.0,9595949.0,5830256.0,3557752.0,3686274.0,NaN,1900037.0,5095689.0,NaN,10459419.0,2073118.0,14820163.0,8009850.0,912532.0,12148485.0,2855176.0,NaN,10920832.0,1048957.0,3303220.0,1103624.0,7469547.0,36129876.0,2784489.0,397691.0,4076636.0,15643017.0,1311930.0,5484375.0,689207.0
6,2747974.0,252970.0,2020702.0,1336910.0,17662374.0,2500334.0,3196187.0,585645.0,8893715.0,3194517.0,1202967.0,605287.0,7362911.0,4324467.0,1407284.0,1190718.0,2113829.0,2612030.0,721180.0,4851065.0,2532057.0,4802420.0,4784552.0,1473102.0,1852953.0,655034.0,601983.0,2233767.0,1025902.0,4965088.0,801689.0,11579156.0,4203608.0,482971.0,5908503.0,1339096.0,1654829.0,10113420.0,633064.0,1644958.0,481465.0,2873537.0,15332333.0,1116797.0,697630.0,3006106.0,4684503.0,1419809.0,2748521.0,173590.0
7,521990.0,153636.0,495034.0,409856.0,10504521.0,816832.0,431196.0,1574576.0,2123462.0,719422.0,275600.0,367178.0,2751479.0,749114.0,954503.0,433229.0,499502.0,393129.0,281155.0,850913.0,1154719.0,1945619.0,1476228.0,492415.0,596167.0,379198.0,196215.0,661240.0,435544.0,1542213.0,322473.0,1771629.0,2197653.0,214463.0,2237002.0,1018657.0,1087449.0,2721979.0,110799.0,582294.0,291896.0,1763747.0,3635024.0,347888.0,126404.0,878751.0,2070931.0,219003.0,1187392.0,187522.0
8,3912800.0,NaN,4545242.0,2866175.0,95152230.0,

В исходном датасете есть информация по очень многим статьям бюджета. Нашему туристическому анализу, конечно же, всё это безобразие не нужно. Поэтому выберем некоторые пункты:

35 - Education

36 - Public welfare

37 - Hospitals

38 - Health

39 - Highways

41 - Police protection

43 - Natural resources

In [39]:
l = [35,36,37,38,39,41,43]
state_budget = df3.loc[l, :].sum()
state_budget

AL     25478449.0
AK      7787939.0
AZ     31085948.0
AR     18392425.0
CA    273846500.0
CO     26845607.0
CT     16748364.0
DE      7386427.0
FL     72781303.0
GA     39795363.0
HI      8344532.0
ID      7504090.0
IL     52589249.0
IN     34338557.0
IA     18884923.0
KS     16345991.0
KY     27053751.0
LA     24870509.0
ME      6794970.0
MD     32919526.0
MA     43053902.0
MI     57608961.0
MN     37314256.0
MS     15877931.0
MO     26072050.0
MT      5363837.0
NE      8336905.0
NV     11487435.0
NH      5782853.0
NJ     50990008.0
NM     14916529.0
NY    143435971.0
NC     47535101.0
ND      5001967.0
OH     62182488.0
OK     18543712.0
OR     27207810.0
PA     77062178.0
RI      6306890.0
SC     24978721.0
SD      3821779.0
TN     26680926.0
TX    124781208.0
UT     17241888.0
VT      5649199.0
VA     42649539.0
WA     44459669.0
WV     10753453.0
WI     29814747.0
WY      3916294.0
dtype: float64

###Анализ задачи

In [40]:
state_budget_map = state_budget.to_dict();

state_budget_column = [population_map[i] for i in reviews_by_states.index.array];

state_budget_and_reviews = pd.DataFrame({'reviews_amount': reviews_by_states.values, 'budget': state_budget_column})
state_budget_and_reviews.index = reviews_by_states.index;
state_budget_and_reviews

,reviews_amount,budget
province,,
AK,21,500653
AR,33,1991649
AZ,212,5691895
CA,1036,32919180
CO,134,4222032
CT,30,1382875
DE,56,273786
FL,854,10680862
GA,281,4629537


Рассмотрим, коррелируют ли данные показатели

In [41]:
state_budget_and_reviews.corr().style.format("{:.2}").background_gradient(cmap='coolwarm', axis=1)

,reviews_amount,budget
reviews_amount,1.0,0.53
budget,0.53,1.0


Аналогично с предыдущей задачей мы видим, что корреляция имеется

Изобразим на графике данные для наглядности

In [42]:
reviewsAndBudgetPlotY = state_budget_and_reviews.copy().sort_values('budget')['reviews_amount'].values;
reviewsAndBudgetPlotX = state_budget_and_reviews.sort_values('budget')['budget'].values;

reviews_population_plot = px.line(x = reviewsAndBudgetPlotX, y = reviewsAndBudgetPlotY, labels={'x':'Бюджет штата', 'y':'Количество отзывов'}, text=state_budget_and_reviews.index)
reviews_population_plot.update_layout(
    margin=dict(l=0, r=30, t=30, b=0),
    title="Зависимость количества отзывов в штате от его бюджета",
    xaxis_title="Бюджет штата",
    yaxis_title="Количество отзывов",
    font_family="Courier New",
    font_color="blue",
    title_font_family="Times New Roman",
    title_font_color="green"
    )

reviews_population_plot.show();

## Проверка статистических гипотез

Проверим гипотезу о средней оценке отелей по Америке:

In [64]:
#Предполагаемая средняя оценка отелей
hypothetical_mean = 4.05;

print('H0: Средняя оценка отзывов равна %.3f' % hypothetical_mean)

print('H1: Средняя оценка отзывов не равна %.3f\n' % hypothetical_mean)

#Вычисляем вероятность того, что среднее генеральной совокупности равняется гипотетическому средниму при условии, что выборка взята из этой генеральной совокупности
stat, p = sc.stats.ttest_1samp(entries['reviews.rating'].values, hypothetical_mean)
print('Statistics=%.3f, p-value=%.3f' % (stat, p))

#Задаём уровень значимости и проверяем, можем ли мы отклонить нулевую гипотезу
alpha = 0.05
if p > alpha:
    print('Принять гипотезу о среднем')
else:
    print('Отклонить гипотезу о среднем')

H0: Средняя оценка отзывов равна 4.050
H1: Средняя оценка отзывов не равна 4.050

Statistics=-1.520, p-value=0.129
Принять гипотезу о среднем


Как мы можем заметить, средний отзыв об отелях в США чуть выше четырёх

#Выводы


* Последний раз данные обновлялись в 2018 году. Это не слишком критично, но всё же для более точного анализа стоит получить более свежие данные.